In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib import animation
from scipy import signal
import scipy.ndimage.filters as nd
import h5py
import pdb
import scipy
import matplotlib

In [2]:
movie_folder = r'/home/william/Documents/011_nice_peaks/'
movie_file = movie_folder[:-1]+'.hdf5'

In [3]:
def playMovie(arr): #arr
    assert len(arr.shape) == 3
    fig = plt.figure()
    im = plt.imshow(arr[:,:,0],interpolation="None")
    def step(i):
        im.set_data(arr[:,:,i])
        plt.title(i)
        return im
    anim = animation.FuncAnimation(fig,step,frames=arr.shape[-1],interval=100)
    plt.show()

In [4]:
def saveMovie(directory):
    files = sorted(os.listdir(directory))

    x,y = np.array(pd.read_csv(directory+files[0],header=None)).shape
    movie_length = len(files)
    
    movie = np.zeros((x,y,movie_length))
    for n in np.arange(movie_length):
        filename = directory+files[n]
        image = np.array(pd.read_csv(filename,header=None))
        if n % 1000 == 0:
            print(n)
        movie[:,:,n] = image
    
    print('Just a second now')
    f = h5py.File(directory[:-1]+'.hdf5',"w")
    f.create_dataset("ECmovie",data=movie)
    f.close()
    print('Done')

In [5]:
def hdf5_to_array(file):
    f = h5py.File(file,'r')
    if f['ECmovie'].shape[-1] > 10000:
        return np.array(f['ECmovie'][:,:,::10])
    return np.array(f['ECmovie'])

In [6]:
def hdf5_to_1d(file):
    f = h5py.File(file,'r')
    return np.mean(np.mean(f['ECmovie'],axis=0),axis=0)

In [7]:
def drawContours(im):
    plt.subplot(121)
    plt.imshow(im,aspect=1)
    plt.axis('equal')
    plt.subplot(122)
    plt.contour(im,15,origin='image')
    plt.axis('equal')
    plt.show()
    

In [8]:
def reduceIm(im,smoothness):
    new_im = scipy.ndimage.correlate(im,np.ones((smoothness,smoothness)))
    return new_im

In [9]:
#Cool peak
f = h5py.File(movie_file,'r')
short_movie = np.array(f['ECmovie'][:,:,18680:18760])

In [10]:
#Hot peak
f = h5py.File(movie_file,'r')
short_movie = np.array(f['ECmovie'][:,:,8542:8602])

In [81]:
avg = np.mean(np.mean(short_movie,axis=0),axis=0)

In [83]:
plt.plot(avg)
plt.show()

In [50]:
movie = hdf5_to_array(movie_file)

movie = movie[:,:,1:]

In [12]:
mean_movie = hdf5_to_1d(movie_file)

In [75]:
plt.imshow(frame1)
plt.show()

In [53]:
mean_movie = mean_movie[1:]

In [ ]:
plt.plot(mean_movie)
plt.show()

In [ ]:
playMovie(corrected_movie)

In [109]:
#Smooth in time in case jiggling still there
smoothness = 3
kernel = np.ones((1,1,smoothness))/smoothness
corrected_movie = nd.correlate(movie,kernel)#[:,:,::smoothness]
smoothness = 10 * smoothness
#kernel2 = np.ones((5,5,1))/25
#corrected_movie = nd.correlate(corrected_movie,kernel)

In [11]:
#cut = np.copy(corrected_movie)
#cut = np.copy(cut[:,:,::smoothness//10])
#cut = cut[:,:,227:1542]

#cut = np.copy(corrected_movie)
cut = np.copy(short_movie)

cut_avg = np.mean(np.mean(cut,axis=0),axis=0)
#all_t = np.arange(cut_avg.size)/ (1000/smoothness)
all_t = np.arange(cut_avg.size)

image_pos = 222
reference_pos = 221
plot_pos = 212

fig = plt.figure(figsize=(17,10))


label_size = 14
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size

plt.subplot(image_pos) #Image
frame1 = cut[:,:,0]
im = plt.imshow(frame1*0,vmin=-1,vmax=1)
plt.axis('off')
cb = plt.colorbar()
cb.set_label('Temperature change / K',fontsize='x-large')
cb.set_ticks([-1,-0.5,0,0.5,1])

plt.subplot(plot_pos) #Plot

ax = plt.gca()

ax.get_yaxis().get_major_formatter().set_useOffset(False)
ax.get_xaxis().set_ticks_position('none')
ax.get_yaxis().set_ticks_position('none')

plt.plot(all_t,cut_avg)

plt.xlim([0,all_t[-1]])
plt.ylabel('Temperature / $\degree$C',fontsize='x-large')
plt.xlabel('time / ms',fontsize='x-large')

line, = plt.plot([0, 0], plt.ylim())
y_range = plt.ylim()

plt.subplot(reference_pos)
plt.imshow(cut[:,:,0])
plt.axis('off')


def animate(i):
    t = all_t[i]
    plt.subplot(image_pos) #Image
    new_im = cut[:,:,i]-frame1
    im.set_data(new_im)

    #plt.title('idx = '+str(idx))
    plt.subplot(plot_pos) #Plot
    line.set_data([t, t], y_range)
    
    return im, line

anim = animation.FuncAnimation(fig, animate, frames=cut.shape[-1], interval=30)
plt.show()

In [170]:
plt.show()

In [168]:
anim.save(save_file,dpi=100,extra_args=['-vcodec', 'libx264'],fps=30)

In [149]:
plt.imshow(frame1)
plt.show()

In [166]:
save_file = r'/home/william/Documents/cool_peak.mp4'

In [ ]:
im1 = cut[:,:,631] - frame1

In [ ]:
plt.imshow(im1,vmin=-1,vmax=1)
plt.colorbar()
plt.axis('off')
plt.show()

In [ ]:
selected_ind = np.array([60,88,400,426,730])

In [ ]:
save_dir = r'/home/william/Dropbox/Materials Internship/Presentations/Video presentation/'

In [ ]:
t = np.arange(cut_avg.size) * (30/1000)
plt.plot(t,cut_avg)
ax = plt.axes()
([plt.plot(t[ind],cut_avg[ind],'o',markersize=10) for ind in selected_ind])
plt.xlim([0,t[754]])
plt.ylim([np.min(cut_avg)*0.999,np.max(cut_avg)*1.001])
plt.ylabel('Temperature / $\degree$ C')
plt.xlabel('Time / s')
y_formatter = matplotlib.ticker.ScalarFormatter(useOffset=False)
ax.yaxis.set_major_formatter(y_formatter)
plt.title('Temperature profile of PMN-PT 011 under cyclic electric field')
plt.show()

In [ ]:
for n in selected_ind:
    im1 = cut[:,:,n] - frame1
    plt.imshow(im1,vmin=-1,vmax=1)
    cb = plt.colorbar()
    plt.axis('off')
    plt.title('Temperature change at t = '+str(t[n])+' s')
    cb.set_label('Temperature change / $\degree$C')
    plt.savefig(save_dir+'image'+str(t[n])+'.png',dpi=300)
    plt.show()
    

In [ ]:
playMovie(movie)

In [ ]:
plt.imshow(frame1)
cb = plt.colorbar()
plt.axis('off')
plt.title('Reference image at t = 0 s')
cb.set_label('Temperature / $\degree$C')
plt.savefig(save_dir+'reference.png',dpi=300)
plt.show()


In [ ]:
box1_avg = np.mean(np.mean(movie,axis=0),axis=0)
box1_avg = np.correlate(box1_avg,np.ones(10)/10)

In [ ]:
plt.plot(box1_avg)
plt.show()

In [ ]:
first_im = movie[:,:,0]
black_box = np.copy(first_im)
black_box[40:60,90:110] = 0

In [ ]:
plt.imshow(black_box)
plt.show()

In [ ]:
rfs

In [ ]:
reduceIm(expandIm(movie[:,:,0])).shape

In [ ]:
a = np.empty(movie.shape)

In [ ]:
movie.shape

In [ ]:
first_im.shape

In [ ]:
a = np.array([[1,2,3],[2,10,6],[9,99,-10]])

In [ ]:
a

In [ ]:
np.unravel_index(np.argmax(a),a.shape)

In [ ]:
first_im.shape

In [ ]:
image = np.zeros((50,50))
image[20:30,30:40] = 1
image.shape

In [ ]:
margin = 10
width = 30
height = 30
image2 = image[margin:margin+height,margin:margin+width]

In [ ]:
corr = scipy.signal.correlate2d(image,image2,mode='valid')

In [ ]:
x,y = np.unravel_index(np.argmax(corr),corr.shape)

In [ ]:
np.all(image[x:x+height,y:y+width] == image2)

In [ ]:
image2.shape

In [ ]:
first_im.shape

In [ ]:
from scipy import signal
from scipy import misc
face = misc.face(gray=True) - misc.face(gray=True).mean()
template = np.copy(face[300:365, 670:750])  # right eye
template -= template.mean()
face = face + np.random.randn(*face.shape) * 50  # add noise
corr = signal.correlate2d(face, template, boundary='symm', mode='same')
y, x = np.unravel_index(np.argmax(corr), corr.shape)  # find the match

import matplotlib.pyplot as plt
fig, (ax_orig, ax_template, ax_corr) = plt.subplots(3, 1,figsize=(6, 15))
ax_orig.imshow(face, cmap='gray')
ax_orig.set_title('Original')
ax_orig.set_axis_off()
ax_template.imshow(template, cmap='gray')
ax_template.set_title('Template')
ax_template.set_axis_off()
ax_corr.imshow(corr, cmap='gray')
ax_corr.set_title('Cross-correlation')
ax_corr.set_axis_off()
ax_orig.plot(x, y, 'ro')
plt.show()